In [1]:
import os
import numpy as np
import pandas as pd
import re

from utils.logger import LOGPATH

In [17]:
fpath = LOGPATH.joinpath('summary-fb.csv')
tori = pd.read_csv(fpath, engine='python')
tori

,seed,data,model,conv,epoch_best,epoch_learn,time_learn,mem_ram_train,mem_cuda_train,f1macro_train,f1micro_train,f1macro_val,f1micro_val,f1macro_test,f1micro_test,time_eval,mem_ram_eval,mem_cuda_eval
0,20,cora,Iterative,AdjConv,63,500,4.1765,1.080,0.059,92.248,93.169,88.792,90.037,87.530,89.094,0.0011,1.080,0.059
1,21,cora,Iterative,AdjConv,45,500,4.0485,1.128,0.059,75.728,85.231,73.496,83.764,72.777,81.331,0.0011,1.128,0.059
2,22,cora,Iterative,AdjConv,50,500,4.0349,1.132,0.059,76.519,85.908,70.762,79.705,76.251,86.876,0.0011,1.132,0.059
3,20,cora,Iterative,ChebConv,47,500,6.4711,1.100,0.071,99.386,99.508,69.611,73.432,73.348,75.416,0.0017,1.100,0.071
4,21,cora,Iterative,ChebConv,46,500,6.2731,1.147,0.071,98.865,99.077,72.122,75.461,73.184,75.231,0.0017,1.147,0.071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,20,twitch-gamer,DecoupledFixedCompose,"Adji2Conv,Adji2Conv-gaussian,gaussian",1,500,314.9026,1.237,4.261,49.212,96.896,49.239,97.002,49.230,96.966,0.0071,1.237,4.261
1102,22,ogbn-mag,DecoupledVar,ChebConv,231,500,265.4695,2.739,15.549,8.313,24.973,6.914,24.134,6.969,25.210,0.0142,2.739,15.549
1103,21,ogbn-mag,DecoupledFixedCompose,"AdjiConv,AdjiConv-mono,mono",202,500,514.0678,2.685,15.231,6.401,26.754,5.891,27.143,5.959,28.482,0.0126,2.685,15.231
1104,21,twitch-gamer,DecoupledFixedCompose,"Adji2Conv,Adji2Conv-gaussian,gaussian",7,500,312.4951,1.685,4.261,49.225,96.948,49.214,96.904,49.215,96.910,0.0076,1.685,4.261


In [18]:
dlst = ["cora", "citeseer", "pubmed", "flickr", "chameleon_filtered", "squirrel_filtered", "actor", "roman_empire", \
        "amazon_ratings", "minesweeper", "tolokers", "questions", "reddit", "penn94", \
        "ogbn-arxiv", "arxiv-year", "genius", "twitch-gamer", "ogbn-mag"]
id_cols = ["seed", 'data', 'model', 'conv',]
m_cols = ['epoch_learn', 'time_learn', 'mem_ram_train', 'mem_cuda_train', 's_f1i_test', 'time_eval',]
t = tori[tori['data'].isin(dlst)][id_cols+m_cols].sort_values(by=['data', 'model', 'conv', 'seed'])
t.sort_index(ascending=False).groupby(['data', 'model', 'conv']).filter(lambda x: len(x) > 3)

,seed,data,model,conv,epoch_learn,time_learn,mem_ram_train,mem_cuda_train,f1micro_test,time_eval


In [19]:
conv_repr = {
    'MLP': {
        'Identity': 'Identity',
    },
    'DecoupledFixed': {
        'AdjConv-impulse':  'Impulse',
        'AdjConv-mono':     'Monomial',
        'AdjConv-appr':     'PPR',
        'AdjConv-hk':       'HK',
        'AdjConv-gaussian': 'Gaussian',
    },
    'DecoupledVar': {
        'AdjConv':          'Var-Monomial',
        'HornerConv':       'Horner',
        'ChebConv':         'Chebyshev',
        'ChebConv2':        'ChebInterp',
        'ClenshawConv':     'Clenshaw',
        'BernConv':         'Bernstein',
    },
    'AdaGNN': {'AdaConv':   'AdaGNN'},
    'ACMGNN': {
        # 'ACMConv-1.0-low-high':     'FBGNNI',
        # 'ACMConv-2.0-low-high':     'FBGNNII',
        # 'ACMConv-1.0-low-high-id':  'ACMGNNI',
        # 'ACMConv-2.0-low-high-id':  'ACMGNNII',
        'ACMConv-2.0-low-high-id':  'ACMGNN',
    },
    'DecoupledFixedCompose': {
        'AdjiConv,AdjiConv-mono,mono': 'FAGNN',
        'Adji2Conv,Adji2Conv-gaussian,gaussian': 'G$^2$CN',
        'AdjDiffConv,AdjDiffConv-appr,appr': 'GNN-LF/HF',
    },
    'DecoupledVarCompose': {
        'AdjConv,ChebConv,BernConv': 'FiGURe',
    }
}
flst = []
for m in conv_repr.values():
    for c, name in m.items():
        flst.append(name)

df = t.groupby(['data', 'model', 'conv'])[m_cols].agg(['mean', 'std'])
df.columns = ['_'.join(col).strip() for col in df.columns.values]
df['time_learn_mean'] = df['time_learn_mean'] * 1000 / df['epoch_learn_mean']
df['time_learn_std'] = df['time_learn_std'] * 1000 / df['epoch_learn_mean']
df['time_eval_mean'] = df['time_eval_mean'] * 1000
df['time_eval_std'] = df['time_eval_std'] * 1000
df.drop(columns=['epoch_learn_mean', 'epoch_learn_std'], inplace=True)
df.reset_index(inplace=True)

df['name'] = df.apply(lambda row: conv_repr[row['model']][row['conv']] if (row['model'] in conv_repr) and (row['conv'] in conv_repr[row['model']]) else None, axis=1)
df.dropna(subset=['name'], inplace=True)
df.drop(columns=['model', 'conv'], inplace=True)
df = df[['data', 'name'] + df.columns.tolist()[1:-1]]
df

,data,name,time_learn_mean,time_learn_std,mem_ram_train_mean,mem_ram_train_std,mem_cuda_train_mean,mem_cuda_train_std,f1micro_test_mean,f1micro_test_std,time_eval_mean,time_eval_std
3,actor,ACMGNN,42.606200,1.100886,1.230000,0.058847,0.413000,0.000000,32.675667,2.957597,7.400000,0.264575
4,actor,AdaGNN,10.009533,1.578104,1.170667,0.049075,0.141000,0.000000,38.070333,0.833196,1.466667,0.115470
5,actor,PPR,9.703267,0.443809,1.130667,0.049075,0.108000,0.000000,37.236667,0.840125,1.300000,0.173205
6,actor,Gaussian,9.855933,1.504255,1.151667,0.057274,0.109000,0.000000,37.763333,0.174053,1.400000,0.200000
7,actor,HK,9.421800,1.131418,1.120000,0.050269,0.108000,0.000000,35.679667,0.665349,1.100000,0.173205
...,...,...,...,...,...,...,...,...,...,...,...,...
364,twitch-gamer,ChebInterp,423.178467,0.837888,1.564000,0.283265,2.687667,0.000577,90.356667,11.306833,12.666667,0.665833
365,twitch-gamer,Clenshaw,165.872400,2.069527,1.584333,0.302829,2.587000,0.000000,97.600000,0.113925,11.033333,0.750555
366,twitch-gamer,Horner,161.500867,2.337176,1.565667,0.284776,2.685667,0.000577,97.015000,0.108171,9.700000,0.866025
367,twitch-gamer,FiGURe,1214.997800,5.551583,1.585333,0.303520,14.454667,0.000577,97.184333,0.427580,333.833333,0.642910


## Latex - Acc

In [20]:
def calc_ranks(mean, std, ascending=False, exc=[]):
    rank = pd.Series(np.nan, index=mean.index)
    na_indices = mean[mean.isna()].index
    nrows = len(mean)
    rank[na_indices] = nrows

    mean = mean.dropna()
    mean = mean[~mean.index.isin(exc)]
    std = std.loc[mean.index]
    idx_pend = mean.sort_values(ascending=ascending).index
    current_rank = 1
    while not idx_pend.empty and current_rank <= nrows:
        idx_top = idx_pend[0]
        top_mean = mean[idx_top]
        top_std = std[idx_top]
        rank[idx_top] = current_rank

        if ascending:
            idx_same = idx_pend[mean[idx_pend] <= top_mean + top_std]
        else:
            idx_same = idx_pend[mean[idx_pend] >= top_mean - top_std]
        rank[idx_same] = current_rank

        idx_pend = idx_pend.difference(idx_same)
        idx_pend = mean.loc[idx_pend].sort_values(ascending=ascending).index
        current_rank += len(idx_same)

    return rank

rk = ['a', 'b', 'c']

In [24]:
# dlst_acc = dlst
# dlst_acc = ["cora", "citeseer", "pubmed", "flickr", "chameleon_filtered", "squirrel_filtered", "actor", "roman_empire"]
# dlst_acc = ["cora", "citeseer", "pubmed", "flickr", "reddit", "chameleon_filtered", "squirrel_filtered", "actor", "roman_empire"]
dlst_acc = ["ogbn-arxiv", "amazon_ratings", "minesweeper", "tolokers", "questions", "penn94", "arxiv-year", "genius", "twitch-gamer"]

acc_str = pd.DataFrame(index=flst, columns=dlst_acc, dtype=str)
for row in acc_str.index:
    for col in acc_str.columns:
        mean = df.loc[(df['name'] == row) & (df['data'] == col), 's_f1i_test_mean'].values
        std = df.loc[(df['name'] == row) & (df['data'] == col), 's_f1i_test_std'].values
        if len(mean) > 0:
            acc_str.loc[row, col] = f'{mean[0]:.2f}\\tpm{{{std[0]:.2f}}}'

# acc_str[acc_str.isna().any(axis=1)]
# acc_str.dropna(inplace=True)
acc_str.fillna('(OOM)', inplace=True)

for col in acc_str.columns:
    mean = df.loc[df['data'] == col, ['name', 's_f1i_test_mean']].set_index('name')['s_f1i_test_mean'].astype(float)
    std  = df.loc[df['data'] == col, ['name', 's_f1i_test_std']].set_index('name')['s_f1i_test_std'].astype(float)
    # rank = calc_ranks(mean, std)
    rank = mean.rank(ascending=False).astype(int)
    for i, rki in enumerate(rk):
        idx = rank[rank == i+1].index
        acc_str.loc[idx, col] = f'\\rk{rki}' + '{' + acc_str.loc[idx, col] + '}'

acc_str

,ogbn-arxiv,amazon_ratings,minesweeper,tolokers,questions,penn94,arxiv-year,genius,twitch-gamer
Identity,50.82\tpm{0.41},43.29\tpm{1.21},80.53\tpm{0.87},78.32\tpm{0.58},96.93\tpm{0.17},74.61\tpm{0.50},35.29\tpm{0.16},80.04\tpm{0.10},\rka{99.39\tpm{0.19}}
Impulse,59.74\tpm{3.56},43.34\tpm{0.51},80.20\tpm{0.70},78.73\tpm{0.59},96.96\tpm{0.03},56.61\tpm{0.70},36.11\tpm{0.68},80.55\tpm{0.40},96.94\tpm{0.03}
Monomial,57.01\tpm{0.17},41.64\tpm{0.50},77.28\tpm{5.05},78.70\tpm{0.90},96.97\tpm{0.06},75.85\tpm{0.30},45.18\tpm{0.38},80.50\tpm{0.14},97.42\tpm{0.42}
PPR,60.89\tpm{1.39},\rkc{43.97\tpm{0.81}},80.53\tpm{0.87},78.25\tpm{0.60},94.35\tpm{4.48},74.97\tpm{0.17},45.55\tpm{0.37},80.78\tpm{0.24},97.18\tpm{0.43}
HK,62.66\tpm{0.57},41.98\tpm{0.21},80.50\tpm{0.74},\rkb{78.92\tpm{0.79}},96.95\tpm{0.06},63.48\tpm{10.13},\rkb{46.82\tpm{0.17}},\rka{81.05\tpm{0.69}},97.34\tpm{0.62}
Gaussian,65.22\tpm{1.05},43.38\tpm{1.14},81.00\tpm{1.61},78.34\tpm{0.59},\rkb{96.98\tpm{0.08}},75.56\tpm{0.17},45.88\tpm{0.53},80.78\tpm{0.17},97.26\tpm{0.55}
Var-Monomial,\rkc{68.50\tpm{0.70}},42.75\tpm{0.53},79.48\tpm{3.62},78.59\tpm{0.69},\rkc{96.97\tpm{0.07}},78.95\tpm{2.89},42.17\tpm{0.59},80.61\tpm{0.46},97.22\tpm{0.49}
Horner,66.00\tpm{0.17},41.67\tpm{0.91},\rkb{85.48\tpm{0.42}},78.24\tpm{0.62},96.96\tpm{0.03},80.17\tpm{3.48},\rkc{46.79\tpm{0.54}},\rkc{80.81\tpm{0.40}},97.01\tpm{0.11}
Chebyshev,52.26\tpm{0.59},40.17\tpm{2.03},75.28\tpm{0.52},78.24\tpm{0.62},96.96\tpm{0.03},75.54\tpm{0.41},35.34\tpm{0.29},79.91\tpm{0.76},97.21\tpm{0.42}
ChebInterp,\rkb{68.76\tpm{0.59}},\rka{45.03\tpm{0.40}},81.37\tpm{1.08},\rka{79.13\tpm{0.67}},\rka{97.03\tpm{0.07}},73.72\tpm{15.51},46.25\tpm{0.45},80.04\tpm{0.09},90.36\tpm{11.31}


In [25]:
for row in acc_str.index:
    print("\t& " + row + " & " + " & ".join(acc_str.loc[row].values) + " \\\\")
print()

	& Identity & 50.82\tpm{0.41} & 43.29\tpm{1.21} & 80.53\tpm{0.87} & 78.32\tpm{0.58} & 96.93\tpm{0.17} & 74.61\tpm{0.50} & 35.29\tpm{0.16} & 80.04\tpm{0.10} & \rka{99.39\tpm{0.19}} \\
	& Impulse & 59.74\tpm{3.56} & 43.34\tpm{0.51} & 80.20\tpm{0.70} & 78.73\tpm{0.59} & 96.96\tpm{0.03} & 56.61\tpm{0.70} & 36.11\tpm{0.68} & 80.55\tpm{0.40} & 96.94\tpm{0.03} \\
	& Monomial & 57.01\tpm{0.17} & 41.64\tpm{0.50} & 77.28\tpm{5.05} & 78.70\tpm{0.90} & 96.97\tpm{0.06} & 75.85\tpm{0.30} & 45.18\tpm{0.38} & 80.50\tpm{0.14} & 97.42\tpm{0.42} \\
	& PPR & 60.89\tpm{1.39} & \rkc{43.97\tpm{0.81}} & 80.53\tpm{0.87} & 78.25\tpm{0.60} & 94.35\tpm{4.48} & 74.97\tpm{0.17} & 45.55\tpm{0.37} & 80.78\tpm{0.24} & 97.18\tpm{0.43} \\
	& HK & 62.66\tpm{0.57} & 41.98\tpm{0.21} & 80.50\tpm{0.74} & \rkb{78.92\tpm{0.79}} & 96.95\tpm{0.06} & 63.48\tpm{10.13} & \rkb{46.82\tpm{0.17}} & \rka{81.05\tpm{0.69}} & 97.34\tpm{0.62} \\
	& Gaussian & 65.22\tpm{1.05} & 43.38\tpm{1.14} & 81.00\tpm{1.61} & 78.34\tpm{0.59} & \rkb{96.98

## Latex - Eff

In [34]:
mlst = ["time_learn", "time_eval", "mem_cuda_train"]
dlst_eff = ["ogbn-arxiv", "amazon_ratings", "minesweeper", "tolokers", "questions", "penn94", "arxiv-year", "genius", "twitch-gamer"]
eff_str = pd.DataFrame(index=flst, columns=pd.MultiIndex.from_product([dlst_eff, mlst]))

for row in eff_str.index:
    for data, met in eff_str.columns:
        mean = df.loc[(df['name'] == row) & (df['data'] == data), met+'_mean'].values
        std = df.loc[(df['name'] == row) & (df['data'] == data), met+'_std'].values
        if len(mean) > 0:
            eff_str.loc[row, (data, met)] = f"{mean[0]:.2f}"
        else:
            eff_str.loc[row, (data, met)] = "-"

for data,met in eff_str.columns:
    mean = df.loc[df['data'] == data, ['name', met+'_mean']].set_index('name')[met+'_mean'].astype(float)
    std  = df.loc[df['data'] == data, ['name', met+'_std']].set_index('name')[met+'_std'].astype(float)
    rko = ['Identity']
    rank = calc_ranks(mean, std, True, rko)
    for i, rki in enumerate(rk):
        idx = rank[rank == i+1].index
        eff_str.loc[idx, (data, met)] = f'\\rk{rki}' + '{' + eff_str.loc[idx, (data, met)] + '}'
    eff_str.loc[rko, (data, met)] = '\\rko{' + eff_str.loc[rko, (data, met)].astype(str) + '}'

eff_str

ogbn-arxiv                            amazon_ratings  \
                time_learn   time_eval mem_cuda_train     time_learn   
Identity       \rko{10.37}  \rko{2.37}     \rko{0.56}     \rko{4.02}   
Impulse       \rka{103.47}  \rkb{4.70}     \rka{1.45}    \rka{12.00}   
Monomial      \rka{103.56}  \rkb{4.67}     \rka{1.45}    \rka{11.52}   
PPR           \rka{104.03}  \rkb{4.57}           1.50    \rka{11.99}   
HK            \rka{104.03}  \rkb{4.63}     \rka{1.45}    \rka{11.12}   
Gaussian            106.44  \rka{3.60}           1.57    \rka{11.69}   
Var-Monomial  \rka{104.69}  \rkb{4.53}           1.53          13.14   
Horner              117.04        9.77           2.54          17.66   
Chebyshev           118.37        7.00           2.48          15.70   
ChebInterp          249.60       13.33           2.56          45.87   
Clenshaw            123.03       10.73           2.54          17.45   
Bernstein           604.27       15.10          11.33          55.54   
AdaGNN              109.75        5.13           2.32          14.44   
ACMGNN              272.45       10.57           8.22          48.15   
FAGNN               224.73        6.40           2.47          21.44   
G$^2$CN             417.21        7.37           4.25          35.11   
GNN-LF/HF           216.75        7.47           2.39          21.21   
FiGURe              804.51      224.97          14.15          75.99   

                                        minesweeper              \
               time_eval mem_cuda_train  time_learn   time_eval   
Identity      \rko{0.90}     \rko{0.10}  \rko{3.44}  \rko{0.60}   
Impulse             2.83     \rkb{0.24}  \rka{8.02}        1.83   
Monomial      \rka{2.50}     \rkb{0.24}  \rka{8.02}  \rka{1.57}   
PPR           \rkc{2.70}     \rkb{0.24}  \rka{7.19}  \rka{1.53}   
HK            \rka{2.37}     \rkb{0.24}  \rka{7.51}  \rka{1.63}   
Gaussian            6.43     \rka{0.24}  \rka{7.95}  \rka{1.63}   
Var-Monomial  \rkc{2.63}     \rkb{0.24}        9.70  \rka{1.57}   
Horner              5.87           0.39       12.60        3.50   
Chebyshev           5.47           0.38       14.16        3.07   
ChebInterp          9.07           0.39       42.67        9.03   
Clenshaw            6.47           0.38       13.58        3.60   
Bernstein          12.17           1.65       32.37        8.67   
AdaGNN              4.67           0.35        9.94        2.00   
ACMGNN             13.47           1.22       38.95        7.73   
FAGNN               5.43           0.39       13.83        3.53   
G$^2$CN             7.30           0.64       17.09        3.80   
GNN-LF/HF           5.60           0.37       12.78        2.83   
FiGURe             25.30           2.07       42.86       12.40   

                                tolokers  ...         penn94   arxiv-year  \
             mem_cuda_train   time_learn  ... mem_cuda_train   time_learn   
Identity         \rko{0.04}   \rko{3.60}  ...     \rko{0.93}   \rko{9.03}   
Impulse          \rkc{0.10}  \rka{21.71}  ...     \rka{1.19}        98.09   
Monomial         \rka{0.10}  \rka{21.94}  ...     \rka{1.19}        95.60   
PPR              \rkc{0.10}  \rka{22.81}  ...     \rka{1.19}  \rka{85.22}   
HK               \rkc{0.10}  \rka{21.93}  ...     \rka{1.19}  \rkc{90.97}   
Gaussian         \rkb{0.10}  \rka{22.47}  ...           1.26        92.50   
Var-Monomial     \rkc{0.10}        23.75  ...     \rka{1.19}  \rka{85.26}   
Horner                 0.15        27.34  ...           1.49        99.48   
Chebyshev              0.15        26.14  ...           1.48        99.83   
ChebInterp             0.16        59.36  ...           1.49       171.22   
Clenshaw               0.15        25.75  ...           1.49       103.50   
Bernstein              0.69       128.71  ...           3.70       499.11   
AdaGNN                 0.15        23.85  ...           1.44  \rkc{90.89}   
ACMGNN                 0.51        64.56  ...           2.88       239.93   
FA

In [40]:
# dlst_eff = ["flickr", "reddit", "tolokers", "penn94"]
dlst_eff = ["penn94", "ogbn-arxiv", "genius", "twitch-gamer"]
for row in eff_str.index:
    values = eff_str.loc[row, dlst_eff].values.tolist()
    while "-" in values:
        idx = values.index("-")
        values[idx] = '\\multicolumn{3}{c|}{(OOM)}'
        for i in range(1, len(mlst)):
            values.pop(idx+1)
    print("\t& " + row + " & " + " & ".join(values) + " \\\\")
print()

	& Identity & \rko{9.03} & \rko{1.93} & \rko{0.93} & \rko{10.37} & \rko{2.37} & \rko{0.56} & \rko{12.21} & \rko{3.03} & \rko{0.95} & \rko{7.97} & \rko{1.97} & \rko{0.56} \\
	& Impulse & \rka{57.11} & \rkb{6.87} & \rka{1.19} & \rka{103.47} & \rkb{4.70} & \rka{1.45} & \rka{115.16} & \rka{6.73} & \rka{3.18} & \rka{149.00} & 4.90 & \rka{1.54} \\
	& Monomial & \rka{56.81} & \rkb{6.83} & \rka{1.19} & \rka{103.56} & \rkb{4.67} & \rka{1.45} & \rka{116.12} & \rka{6.73} & 3.41 & \rka{148.81} & 5.77 & \rka{1.54} \\
	& PPR & \rka{57.21} & \rkb{7.13} & \rka{1.19} & \rka{104.03} & \rkb{4.57} & 1.50 & 116.19 & \rka{6.77} & 3.41 & \rka{148.13} & \rkb{4.17} & \rka{1.54} \\
	& HK & \rka{57.62} & \rkb{6.87} & \rka{1.19} & \rka{104.03} & \rkb{4.63} & \rka{1.45} & \rka{115.53} & 7.10 & \rka{3.18} & \rka{148.15} & \rkb{4.13} & \rka{1.54} \\
	& Gaussian & \rka{56.88} & \rka{6.10} & 1.26 & 106.44 & \rka{3.60} & 1.57 & 116.31 & \rka{6.60} & 3.46 & \rka{148.10} & \rka{3.67} & 1.72 \\
	& Var-Monomial & \rka{57.8

## Latex - Full